In [1]:
from neattext.functions import  remove_emails, fix_contractions, remove_numbers,remove_emojis,remove_special_characters, \
                                remove_stopwords, remove_html_tags, remove_urls, remove_puncts,remove_custom_pattern

from neattext.pipeline import TextPipeline


from transformers import BertTokenizer, BertForSequenceClassification
import transformers

import pandas as pd
import re


In [2]:
# Chat GPT3
def remove_multiple_spaces(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text


def clean_tweet(tweet):
    tweet = remove_emails(tweet)
    tweet = fix_contractions(tweet)
    tweet = remove_emojis(tweet)
    tweet = remove_urls(tweet)
    tweet = remove_html_tags(tweet)
    tweet = remove_custom_pattern(tweet, r'[^a-zA-Z0-9+\-%\&\?\!\.\,\s]')
    tweet = remove_custom_pattern(tweet, r'@\w+|#\w+|Re-pinging')
    tweet = remove_multiple_spaces(tweet)

    return tweet

In [3]:
def remove_specific_patterns(text, pattern):
    return remove_custom_pattern(text, pattern)      

def specific_cleaning(text):
    specific_cleaning = TextPipeline(steps=[remove_emails, fix_contractions, remove_emojis, remove_html_tags, remove_urls])
    return specific_cleaning.fit(text)

def final_cleaning(text, pattern):
    msg = remove_specific_patterns(text, pattern)
    almost_clean_text = specific_cleaning(msg)
    return almost_clean_text





In [67]:
texto = "I'm This is -66% +55 the mail example@gmail.com ,our WEBSITE is https://example.com 😊."
texto = clean_tweet(texto)

texto

'i am this is -66% +55 the mail ,our website is .'

In [44]:
#finbert for sentiment analysis
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')


def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
    return outputs.argmax().item()

In [45]:
def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
    return outputs.argmax().item()

In [46]:
finbert_fls = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
tokenizer_fls = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')


def fls_classification(text):
    inputs = tokenizer_fls(text, return_tensors="pt", padding=True)
    outputs = finbert_fls(**inputs)[0]
    return outputs.argmax().item()

In [4]:
tweets = "C:\\Users\\Borba\\Documents\\GitHub\\stock_price_streamlit_app\\tweets\\stock_market_tweets.csv"

In [6]:
df = pd.read_csv(tweets)

In [7]:
df["body"] = df["body"].apply(lambda x: x.lower())

In [8]:
df_test = df.sample(3000)

In [9]:
df_test["clean_message"] = df["body"].apply(lambda x: clean_tweet(x))

In [10]:
df_test[["body" , "clean_message"]]

,body,clean_message
1613196,earnings coming - excercise caution on er play...,earnings coming - excercise caution on er play...
1109060,did you like the part where elon said $tsla wa...,did you like the part where elon said tsla was...
1212428,the s&p has a 20.6% weighting in its top 10 na...,the s p has a 20.6% weighting in its top 10 na...
1502634,happy belated $tsla earnings miss everyone!,happy belated tsla earnings miss everyone
1330525,$amzn i did go into nvda by doubling the posit...,amzn i did go into nvda by doubling the posit...
...,...,...
842764,remember the last time @nba finals did not fea...,remember the last time nba finals did not feat...
1250850,$googl calls above $1098.20,googl calls above 1098.20
116325,#sentishiftup $ibm $spx $uup $es $ndx $xom $ms...,sentishiftup ibm spx uup es ndx xom msft dis ...
475539,#microsoft $msft doubles #azure #sql database ...,microsoft msft doubles azure sql database rea...


In [53]:
df_test["sentiment_analysis"] = df_test["clean_message"].apply(lambda x: sentiment_analysis(x))

<ipython-input-53-06edbe03d645>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["sentiment_analysis"] = df_test["clean_message"].apply(lambda x: sentiment_analysis(x))


In [54]:
df_test["fls_classification"] = df_test["clean_message"].apply(lambda x: fls_classification(x))

<ipython-input-54-af7108ce3146>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["fls_classification"] = df_test["clean_message"].apply(lambda x: fls_classification(x))


In [32]:
#salvar o arquivo para não precisar rodar novamente
df_test.to_csv("C:\\Users\\Borba\\Documents\\GitHub\\stock_price_streamlit_app\\tweets\\stock_market_tweets_clean_test.csv", index=False)


In [2]:
df_test = pd.read_csv("C:\\Users\\Borba\\Documents\\GitHub\\stock_price_streamlit_app\\tweets\\stock_market_tweets_clean_test.csv")

In [17]:
df_test[181:182]

,Unnamed: 0,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,clean_message,Sentiment
181,1035400,1030319910063366145,PlugInFUD,2018-08-17,"sec investing multiple things, internal counse...",1,0,0,TSLA,"sec investing multiple things, internal counse...",NaN


In [15]:
index = 0

In [12]:
# df_test['Sentiment'] = None
# df_test['Relevant'] = None

# Iterate through the DataFrame
for index, row in df_test[181:].iterrows():
    # Read the text from the "Text" column
    text = row['clean_message']
    company = row['ticker_symbol']
    body = row['body']
    labels = {0:'neutral', 1:'positive',2:'negative'}


    # Get the user input for the sentiment value
    sentiment = input(f"Enter sentiment for {company}: '{text}': and '{body}' ")
    
    # Get the user input for the relevant value
    # relevant = input(f"Enter relevant value for {company}: '{text}': ")
    if sentiment == 'exit':
        df_test.to_csv("C:\\Users\\Borba\\Documents\\GitHub\\stock_price_streamlit_app\\tweets\\stock_market_tweets_clean_test.csv", index=False)
        break
    # Update the "Sentiment" and "Relevant" columns with the user-provided values
    df_test.at[index, 'Sentiment'] = sentiment
    
    
    # df_test.at[index, 'Relevant'] = relevant

142  Unnamed: 0                                                 1714089
tweet_id                                       1187155754853261312
writer                                              Pivotal_Pivots
post_date                                               2019-10-23
body             $tsla the low this year was on the annual ys2 ...
comment_num                                                      0
retweet_num                                                      4
like_num                                                         4
ticker_symbol                                                 TSLA
clean_message     tsla the low this year was on the annual ys2 ...
Sentiment                                                      NaN
Name: 142, dtype: object
143  Unnamed: 0                                                 1435043
tweet_id                                       1114610982528020480
writer                                             KennethMilesTa2
post_date                  

In [13]:
print(index, '', row)




181  Unnamed: 0                                                 1035400
tweet_id                                       1030319910063366145
writer                                                   PlugInFUD
post_date                                               2018-08-17
body             sec investing multiple things, internal counse...
comment_num                                                      1
retweet_num                                                      0
like_num                                                         0
ticker_symbol                                                 TSLA
clean_message    sec investing multiple things, internal counse...
Sentiment                                                      NaN
Name: 181, dtype: object


In [ ]:
index

In [27]:
df_test[57:58]

,Unnamed: 0,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,clean_message,Sentiment
206233,217960,659335482510331904,WallStreetMainS,2015-10-28,$aapl guidance very strong. will likely beat t...,0,0,0,AAPL,aapl guidance very strong. will likely beat t...,None


In [35]:
num = 0
for ind, row in df_test.iterrows():
    num = num + 1
    if ind == index:
        break
print(f"Total of {num} rows updated.")

Total of 212 rows updated.


In [77]:
def to_int(x):
    if x is None or x == "":
        return 0
    else:
        return int(x)


df_test["Sentiment"] = df_test["Sentiment"].apply(lambda x: to_int(x))

df_test["Count"] = df_test["Sentiment"] == df_test["sentiment_analysis"]

#transform string to int    


<ipython-input-77-e0c653014876>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Sentiment"] = df_test["Sentiment"].apply(lambda x: to_int(x))
<ipython-input-77-e0c653014876>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Count"] = df_test["Sentiment"] == df_test["sentiment_analysis"]


In [79]:
df_test.sum()

<ipython-input-79-6ca408b47255>:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_test.sum()


Unnamed: 0                                                       507986
tweet_id                                           -2296998721481367074
post_date             2015-01-012015-01-012015-01-012015-01-012015-0...
body                  insanity of today weirdo massive selling. $aap...
comment_num                                                         211
retweet_num                                                         242
like_num                                                            609
ticker_symbol         AAPLAAPLAAPLAAPLGOOGAAPLAAPLGOOGAAPLAAPLAAPLMS...
clean_message         insanity of today weirdo massive selling. aapl...
sentiment_analysis                                                  380
fls_classification                                                  199
Sentiment                                                            62
Count                                                               742
dtype: object